<a href="https://colab.research.google.com/github/menduv/mtg-Jamendo/blob/main/CNN_mtg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/colab_data/trail_download_archive_6gb.zip","r") as zip_ref:
    zip_ref.extractall("/content/sample_mel")

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
import os

print('The number of mel-spectrogram in the sample_mel folder is:'.len(os.listdir('/content/sample_mel')))

1991

In [5]:
mel_dir = '/content/sample_mel'
mel_list = os.listdir(mel_dir)
mel_npy = np.load(f'{mel_dir}/{mel_list[25]}')
# print('Count of npy files: ', len(mel_list))
print('Size of image: ', mel_npy.shape)

Size of image:  (96, 12506)


In [7]:
data = pd.read_csv('/content/mtg_cleaned.csv', index_col = [0])

In [8]:
data

,track_id,artist_id,album_id,path,duration,genre,instrument,mood
1,214,14,31,14/214.mp3,124.6,punkrock,NaN,NaN
2,215,14,31,15/215.mp3,151.4,metal,NaN,NaN
3,216,14,31,16/216.mp3,234.9,metal,NaN,NaN
4,217,14,31,17/217.mp3,127.9,punkrock,NaN,NaN
5,218,14,31,18/218.mp3,180.7,punkrock,NaN,NaN
...,...,...,...,...,...,...,...,...
55605,1422056,496314,165847,56/1422056.mp3,516.0,soundtrack,computer,"advertising, dramatic, epic, movie"
55606,1422057,496314,165847,57/1422057.mp3,374.9,soundtrack,computer,"advertising, dramatic, epic, movie"
55607,1422058,496314,165847,58/1422058.mp3,315.8,soundtrack,computer,"dramatic, epic, movie"
55608,1422059,496314,165847,59/1422059.mp3,201.3,soundtrack,computer,"advertising, dramatic, epic, movie"


In [9]:
data.dtypes

track_id        int64
artist_id       int64
album_id        int64
path           object
duration      float64
genre          object
instrument     object
mood           object
dtype: object

In [10]:
track_ids = [int(file_name.replace('.npy', '')) for file_name in mel_list]

In [11]:
df = data[data['track_id'].isin(track_ids)]

In [ ]:
df['genre'] = df['genre'].str.split(',').str[0]

In [13]:
df = df[['track_id', 'genre']]

In [14]:
df

,track_id,genre
522,6606,dance
523,6607,dance
524,6608,dance
525,6609,dance
636,7400,choir
...,...,...
55540,1420702,club
55541,1420704,dance
55542,1420705,dance
55543,1420706,dance


In [15]:
train_track_ids = df['track_id'].values

In [16]:
train_img = []
for ID in train_track_ids:
    img_path = os.path.join(f'{mel_dir}/{ID}.npy')
    img = np.load(img_path)
    train_img.append(img)

In [17]:
y = df['genre'].values

In [18]:
!mkdir '/content/cropped_mel'

In [20]:
def crop_melspectrogram(mel_spectrogram, target_frames, hop_size):
    # Crop or pad the mel spectrogram to the target number of frames
    if mel_spectrogram.shape[1] >= target_frames:
        cropped_mel_spectrogram = mel_spectrogram[:, :target_frames]
    else:
        # Pad with zeros if the original length is less than the target
        padding = np.zeros((mel_spectrogram.shape[0], target_frames - mel_spectrogram.shape[1]))
        cropped_mel_spectrogram = np.hstack((mel_spectrogram, padding))

    return cropped_mel_spectrogram

# Example usage
input_npy_path = '/content/sample_mel/1001307.npy'
output_npy_path = '/content/cropped_mel/1001307.npy'

# Load the mel spectrogram from the .npy file
mel_spectrogram = np.load(input_npy_path)

# Assuming the hop size used during mel spectrogram calculation
hop_size = 256
sample_rate = 12000

# Calculate the target number of frames based on the desired duration (e.g., 30 seconds)
desired_duration_seconds = 30
target_frames = int(desired_duration_seconds * (1 / (hop_size / sample_rate)))

# Crop the mel spectrogram to the target number of frames
cropped_mel_spectrogram = crop_melspectrogram(mel_spectrogram, target_frames, hop_size)

# Save the cropped mel spectrogram as a new .npy file
np.save(output_npy_path, cropped_mel_spectrogram)


In [28]:
folder_path = '/content/sample_mel'
cropped_folder_path = '/content/cropped_mel'

# Assuming the hop size used during mel spectrogram calculation
hop_size = 256
sample_rate = 12000

# Calculate the target number of frames based on the desired duration (e.g., 30 seconds)
desired_duration_seconds = 30
target_frames = int(desired_duration_seconds * (1 / (hop_size / sample_rate)))

# Iterate over files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    mel_spectrogram = np.load(file_path)

    cropped_mel_spectrogram = crop_melspectrogram(mel_spectrogram, target_frames, hop_size)

    np.save(os.path.join(cropped_folder_path, filename), cropped_mel_spectrogram)

In [32]:
print('The number of cropped mel-spectrogram in the cropped_mel folder is:',len(os.listdir('/content/cropped_mel')))

The number of cropped mel-spectrogram in the cropped_mel folder is: 1992


In [34]:
'''To display cropped and original mels'''

# crop_mel_307 = np.load('/content/cropped_mel/1001307.npy')
# mel_307 = np.load('/content/sample_mel/1001307.npy')

# plt.imshow(crop_mel_307, aspect='auto')
# plt.xlabel('Time')
# plt.ylabel('Mel Frequency Bin')
# plt.title('Mel-Spectrogram')
# plt.colorbar()
# plt.show()

# plt.imshow(mel_307, aspect='auto')
# plt.xlabel('Time')
# plt.ylabel('Mel Frequency Bin')
# plt.title('Mel-Spectrogram')
# plt.colorbar()
# plt.show()

In [35]:
crop_mel_307.shape

(96, 1406)

In [36]:
mel_307.shape

(96, 11975)

In [ ]:
#greyscalling and reducing the size significantly!, change when possible
from skimage.transform import resize

target_shape = (96, 96)  # Choose the desired shape

resized_arrays = [resize(img, target_shape, mode='reflect', anti_aliasing=True) for img in train_img]

train_img_resized = np.array(resized_arrays).reshape(-1, target_shape[0], target_shape[1], 1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_img_resized, y, test_size=0.1, random_state=32)

In [ ]:
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import tensorflow as tf

In [ ]:
# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(107, 107,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(5, activation='linear'))

opt = Adam(lr=0.005)
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 107, 107, 64)      640       
                                                                 
 batch_normalization (Batch  (None, 107, 107, 64)      256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 107, 107, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 53, 53, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 53, 53, 64)        0         
                                                                 
 flatten (Flatten)           (None, 179776)            0

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')

callbacks = [reduce_lr]

history = model.fit(
    x=X_train,
    y=y_train,
    epochs=50,
    validation_data = (X_val,y_val),
    callbacks=callbacks
)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Assuming y contains string labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert to one-hot encoding
y_categorical = tf.keras.utils.to_categorical(y_encoded, num_classes=len(label_encoder.classes_))

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_img_resized, y_categorical, test_size=0.1, random_state=32)

# Rest of the code remains unchanged
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(107, 107, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Use 'softmax' activation for multi-class classification
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

opt = Adam(lr=0.005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.00001, mode='auto')

callbacks = [reduce_lr]

history = model.fit(
    x=X_train,
    y=y_train,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)
